In [57]:
# -*- coding: utf-8 -*-
import os
import io
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [60]:
# Параметры последовательностей
maxlen = 40
step = 3

df = pd.read_csv('books.csv', usecols=["Book-Title"], encoding='cp1251')
df = df.dropna().drop_duplicates()
lines = df['Book-Title'].tolist()
lines = [line.strip() for line in lines if line.strip()]
print("Количество исходных названий:", len(lines))

Количество исходных названий: 238708


In [61]:
# Объединяем все названия в один большой текст
text = "\n".join(lines)
print("Общий объём символов в тексте:", len(text))

# Собираем уникальные символы
chars = sorted(list(set(text)))
print("Число уникальных символов:", len(chars))

char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

Общий объём символов в тексте: 9052382
Число уникальных символов: 140


In [64]:
# Формируем обучающие примеры
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])

print("Количество обучающих последовательностей:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Количество обучающих последовательностей: 3017448


In [67]:
# Строим модель
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

# Создадим директорию для сохранения моделей
if not os.path.exists('models'):
    os.mkdir('models')

# Коллбеки
early_stopping = EarlyStopping(
    monitor='loss', 
    patience=40, 
    restore_best_weights=True
)
model_checkpoint = ModelCheckpoint(
    'models/rano_gen_model.keras',
    monitor='loss', 
    save_best_only=True
)

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                        │ (None, 128)                 │         137,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 140)                 │          18,060 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 155,788 (608.55 KB)

 Trainable params: 155,788 (608.55 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
# Функция выборки следующего символа
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [69]:
# Обучение модели и генерация текста после каждой эпохи
epochs = 20
batch_size = 128

for epoch in range(epochs):
    print("Epoch:", epoch + 1, "/", epochs)
    model.fit(
        x, y, 
        batch_size=batch_size, 
        epochs=1, 
        callbacks=[early_stopping, model_checkpoint]
    )
    model.load_weights('models/rano_gen_model.keras')

    # Генерируем текст после эпохи
    print("\nГенерация текста после эпохи:", epoch + 1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    seed = text[start_index : start_index + maxlen]
    print('Сид фрагмент: "' + seed + '"\n')

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("Диверсификация:", diversity)

        generated = ""
        sentence = seed
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                if char in char_indices:
                    x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

        print("Сгенерированный текст:", generated)
        print()

# Загрузка лучшей модели после обучения
model.load_weights('models/rano_gen_model.keras')

Epoch: 1 / 20
23574/23574 ━━━━━━━━━━━━━━━━━━━━ 808s 34ms/step - loss: 2.0108

Генерация текста после эпохи: 1
Сид фрагмент: "Crime Monthly)
Criminal Convictions: Err"

Диверсификация: 0.2
Сгенерированный текст: or (Silhouette Intimate Moments
The Book of Black (Silhouette Intimate Momental Edition (Paperback))
The Book of the World (Silhouette Intimate Moments
The Sea Story (Harlequin Romance)
The Book of the Soul (Harlequin Intrigue
The Book of the Companion: The Contemporary Collection (Harlequin Intrigue
The Book of the Back (Silhouette Intimate Momental Edition
The Sea Short Story (Penguin Series)
Th

Диверсификация: 0.5
Сгенерированный текст: or Book of the Bantasting Good Sychology (Love Special Edition (Paperback))
A Novel (Vintage Press Gombar Library)
Care and Medical Stories (Baby Sinner Mystery)
The Bird of Stail (The Book Book)
The Sheit Book of Montand : The Secret Contemporary Works (Paperback))
Wild Bears of the Moon (Harlequin Intemperiage)
Death of Mart Sea (The Rain
L

KeyboardInterrupt: 